In [ ]:
# -*- coding: utf-8 -*-
# ===== (1) Environment-level reproducibility settings: before importing torch =====
import os
import random

SEED = 20240518  # You can change this to any fixed number
os.environ["PYTHONHASHSEED"] = str(SEED)
# Enable determinism for certain cublas ops (requires CUDA >= 10.2; must be set before first CUDA use)
# Allowed values ":16:8" or ":4096:8" (larger memory). Ignored if unsupported by your environment.
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"

# ===== (2) Common imports =====
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, classification_report, brier_score_loss
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


# ===== (3) Unified global seed setup function =====
def set_global_seed(seed: int = 20240518):
    # Python / NumPy / PyTorch
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # cuDNN / TF32: avoid nondeterminism or numeric drift
    try:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    except Exception:
        pass
    # Disable TF32 (some GPUs/drivers enable it by default, which may introduce small differences)
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

    # Strictly use deterministic algorithms (will throw if an op isn't supported; use warn_only=True for warnings)
    try:
        torch.use_deterministic_algorithms(True)
    except Exception:
        pass


# ===== (4) Device selection (doesn't affect determinism, but affects speed) =====
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


# ===== (5) Probability calibration class =====
class ProbabilityCalibrator:
    """Probability calibrator supporting Platt Scaling and Isotonic Regression"""
    
    def __init__(self, method='platt'):
        """
        Initialize calibrator
        Args:
            method: 'platt' for Platt Scaling, 'isotonic' for Isotonic Regression
        """
        self.method = method
        if method == 'platt':
            self.calibrator = LogisticRegression()
        elif method == 'isotonic':
            self.calibrator = IsotonicRegression(out_of_bounds='clip')
        else:
            raise ValueError("Method must be 'platt' or 'isotonic'")
        
        self.is_fitted = False
    
    def fit(self, probabilities, true_labels):
        """
        Train the calibrator on a calibration set
        Args:
            probabilities: model output probabilities (numpy array)
            true_labels: true labels (numpy array)
        """
        probabilities = np.array(probabilities).reshape(-1, 1)
        true_labels = np.array(true_labels)
        
        self.calibrator.fit(probabilities, true_labels)
        self.is_fitted = True
    
    def predict_proba(self, probabilities):
        """
        Calibrate probabilities
        Args:
            probabilities: probabilities to calibrate (numpy array)
        Returns:
            Calibrated probabilities (numpy array)
        """
        if not self.is_fitted:
            raise ValueError("Calibrator must be fitted before prediction")
        
        probabilities = np.array(probabilities).reshape(-1, 1)
        
        if self.method == 'platt':
            return self.calibrator.predict_proba(probabilities)[:, 1]
        else:  # isotonic
            return self.calibrator.predict(probabilities.flatten())


def evaluate_calibration(y_true, y_prob, n_bins=10):
    """
    Evaluate calibration quality
    Args:
        y_true: true labels
        y_prob: predicted probabilities
        n_bins: number of bins
    Returns:
        dict: a dictionary containing various calibration metrics
    """
    fraction_positives, mean_predicted = calibration_curve(
        y_true, y_prob, n_bins=n_bins
    )
    
    # Compute Expected Calibration Error (ECE)
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]
    
    ece = 0
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        in_bin = (y_prob > bin_lower) & (y_prob <= bin_upper)
        prop_in_bin = in_bin.mean()
        
        if prop_in_bin > 0:
            accuracy_in_bin = y_true[in_bin].mean()
            avg_confidence_in_bin = y_prob[in_bin].mean()
            ece += np.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin
    
    # Brier Score
    brier_score = brier_score_loss(y_true, y_prob)
    
    return {
        'ece': ece,
        'brier_score': brier_score,
        'fraction_positives': fraction_positives,
        'mean_predicted': mean_predicted
    }


# ===== (6) Data loading and preprocessing =====
def load_and_preprocess_data():
    train_df = pd.read_csv('mimicdeadtrain.csv')
    test_df  = pd.read_csv('mimicdeadtest.csv')

    print(f"Train set size: {train_df.shape}")
    print(f"Test set size: {test_df.shape}")
    print(f"Train set positive rate: {train_df['death'].mean():.4f}")
    print(f"Test set positive rate: {test_df['death'].mean():.4f}")

    feature_columns = [c for c in train_df.columns if c != 'death']

    X_train = train_df[feature_columns].values
    y_train = train_df['death'].values.astype(np.float32)
    X_test  = test_df[feature_columns].values
    y_test  = test_df['death'].values.astype(np.float32)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)

    # Three-way split: train -> train (60%) + calibration (20%) + validation (20%)
    X_temp, X_val, y_temp, y_val = train_test_split(
        X_train_scaled, y_train, test_size=0.2, random_state=SEED, stratify=y_train
    )
    
    X_tr, X_cal, y_tr, y_cal = train_test_split(
        X_temp, y_temp, test_size=0.25, random_state=SEED, stratify=y_temp  # 0.25 * 0.8 = 0.2
    )

    # Convert to tensors
    X_tr  = torch.tensor(X_tr,  dtype=torch.float32)
    y_tr  = torch.tensor(y_tr,  dtype=torch.float32)
    X_cal = torch.tensor(X_cal, dtype=torch.float32)
    y_cal = torch.tensor(y_cal, dtype=torch.float32)
    X_val = torch.tensor(X_val, dtype=torch.float32)
    y_val = torch.tensor(y_val, dtype=torch.float32)
    X_te  = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_te  = torch.tensor(y_test, dtype=torch.float32)

    print(f"Sizes after split:")
    print(f"  Train: {X_tr.shape[0]}")
    print(f"  Calibration: {X_cal.shape[0]}")
    print(f"  Validation: {X_val.shape[0]}")
    print(f"  Test: {X_te.shape[0]}")

    return X_tr, y_tr, X_cal, y_cal, X_val, y_val, X_te, y_te, len(feature_columns)


# ===== (7) Model definitions =====
# 7.1 MLP
class MLPModel(nn.Module):
    def __init__(self, input_dim, hidden_dims=[128, 64, 32], dropout=0.3):
        super().__init__()
        layers, prev = [], input_dim
        for h in hidden_dims:
            layers += [nn.Linear(prev, h), nn.BatchNorm1d(h), nn.ReLU(), nn.Dropout(dropout)]
            prev = h
        layers += [nn.Linear(prev, 1)]  # no Sigmoid; use BCEWithLogitsLoss
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x).squeeze(-1)


# 7.2 GRU + Attention
class GRUAttnModel(nn.Module):
    """
    Treat each numerical feature as a 'token':
    token = value_proj(value) + feature_embedding[feature_id]
    Feed tokens to a BiGRU, then use trainable attention for weighted pooling, and finally classify.
    """
    def __init__(self, input_dim, emb_dim=64, hidden_dim=128, num_layers=2,
                 bidirectional=True, dropout=0.3, attn_hidden=128):
        super().__init__()
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.bidirectional = bidirectional
        self.num_directions = 2 if bidirectional else 1

        # Assign a learnable embedding for each feature position
        self.feature_emb = nn.Embedding(input_dim, emb_dim)
        # Scalar -> vector
        self.val_proj = nn.Linear(1, emb_dim)
        self.token_ln = nn.LayerNorm(emb_dim)

        self.gru = nn.GRU(
            input_size=emb_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=bidirectional
        )

        # Attention (additive / HAN-style)
        self.attn_fc = nn.Linear(hidden_dim * self.num_directions, attn_hidden)
        self.attn_tanh = nn.Tanh()
        self.attn_vec = nn.Linear(attn_hidden, 1, bias=False)

        self.dropout = nn.Dropout(dropout)

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * self.num_directions, 64),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)  # no Sigmoid; use BCEWithLogitsLoss
        )

        # Initialization (controlled by global seed)
        nn.init.xavier_uniform_(self.val_proj.weight)
        nn.init.zeros_(self.val_proj.bias)

    def forward(self, x):
        # x: (batch, seq_len)  Treat each feature as a time step
        B, S = x.shape
        feat_ids = torch.arange(S, device=x.device).unsqueeze(0).expand(B, S)  # (B, S)

        value_tokens = self.val_proj(x.unsqueeze(-1))            # (B, S, emb_dim)
        feat_tokens  = self.feature_emb(feat_ids)                # (B, S, emb_dim)
        tokens = self.token_ln(value_tokens + feat_tokens)       # (B, S, emb_dim)
        tokens = self.dropout(tokens)

        H, _ = self.gru(tokens)  # (B, S, hidden*dirs)

        # Attention weights
        U = self.attn_tanh(self.attn_fc(H))          # (B, S, attn_hidden)
        scores = self.attn_vec(U).squeeze(-1)        # (B, S)
        alpha = torch.softmax(scores, dim=1)         # (B, S)
        context = torch.sum(H * alpha.unsqueeze(-1), dim=1)  # (B, hidden*dirs)

        context = self.dropout(context)
        logits = self.classifier(context).squeeze(-1)  # (B,)
        return logits  # logits (before sigmoid)


# 7.3 Transformer
class TransformerModel(nn.Module):
    def __init__(self, input_dim, d_model=64, nhead=8, num_layers=2, dropout=0.3):
        super().__init__()
        self.input_projection = nn.Linear(1, d_model)
        self.pos_encoding = nn.Parameter(torch.randn(input_dim, d_model))  # controlled by global seed
        enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead,
                                               dropout=dropout, batch_first=True)
        self.transformer = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
        self.classifier = nn.Sequential(
            nn.Linear(d_model, 32), nn.ReLU(), nn.Dropout(dropout),
            nn.Linear(32, 1)  # no Sigmoid
        )

    def forward(self, x):
        B, S = x.size(0), x.size(1)
        x = self.input_projection(x.unsqueeze(-1))              # (B, S, d_model)
        x = x + self.pos_encoding.unsqueeze(0).expand(B, -1, -1)
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.classifier(x).squeeze(-1)


# ===== (8) Training and evaluation =====
def get_model_predictions(model, data_loader):
    """Get model predicted probabilities"""
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for xb, yb in data_loader:
            xb = xb.to(device)
            logits = model(xb)
            probs = torch.sigmoid(logits)
            preds.extend(probs.cpu().numpy())
            labels.extend(yb.numpy())
    return np.array(preds), np.array(labels)


def train_model(model, train_loader, val_loader, epochs=100, lr=1e-3, pos_weight=None):
    if pos_weight is not None:
        pos_weight = torch.tensor(pos_weight, dtype=torch.float32, device=device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=10, verbose=False, min_lr=1e-6
    )

    model.to(device)
    best_auc, patience_counter, patience = 0.0, 0, 30
    train_losses, val_aucs = [], []

    for epoch in range(epochs):
        model.train()
        running = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)

            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
            optimizer.step()
            running += loss.item()

        # Validation
        val_preds, val_labels = get_model_predictions(model, val_loader)
        val_auc = roc_auc_score(val_labels, val_preds)
        scheduler.step(val_auc)

        train_losses.append(running / max(1, len(train_loader)))
        val_aucs.append(val_auc)

        improved = val_auc > best_auc + 1e-6
        if improved:
            best_auc = val_auc
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1:03d}/{epochs} | Train Loss: {train_losses[-1]:.4f} | Val AUC: {val_auc:.4f}")

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    model.load_state_dict(torch.load('best_model.pth', map_location=device))
    return model, best_auc, train_losses, val_aucs


def evaluate_model_with_calibration(model, cal_loader, test_loader, model_name):
    """Evaluate the model and perform probability calibration"""
    # Get predictions on the calibration set
    cal_preds, cal_labels = get_model_predictions(model, cal_loader)
    
    # Get raw predictions on the test set
    test_preds, test_labels = get_model_predictions(model, test_loader)
    
    # Train calibrators
    platt_calibrator = ProbabilityCalibrator(method='platt')
    isotonic_calibrator = ProbabilityCalibrator(method='isotonic')
    
    platt_calibrator.fit(cal_preds, cal_labels)
    isotonic_calibrator.fit(cal_preds, cal_labels)
    
    # Calibrate test-set predictions
    test_preds_platt = platt_calibrator.predict_proba(test_preds)
    test_preds_isotonic = isotonic_calibrator.predict_proba(test_preds)
    
    # Compute AUC (calibration should not change AUC)
    auc_original = roc_auc_score(test_labels, test_preds)
    auc_platt = roc_auc_score(test_labels, test_preds_platt)
    auc_isotonic = roc_auc_score(test_labels, test_preds_isotonic)
    
    # Evaluate calibration quality
    cal_metrics_original = evaluate_calibration(test_labels, test_preds)
    cal_metrics_platt = evaluate_calibration(test_labels, test_preds_platt)
    cal_metrics_isotonic = evaluate_calibration(test_labels, test_preds_isotonic)
    
    # Save prediction results
    results_df = pd.DataFrame({
        'y_true': test_labels,
        'y_proba_original': test_preds,
        'y_proba_platt': test_preds_platt,
        'y_proba_isotonic': test_preds_isotonic
    })
    results_df.to_csv(f"death_{model_name}_calibrated_proba.csv", index=False)
    
    return {
        'auc_original': auc_original,
        'auc_platt': auc_platt,
        'auc_isotonic': auc_isotonic,
        'calibration_original': cal_metrics_original,
        'calibration_platt': cal_metrics_platt,
        'calibration_isotonic': cal_metrics_isotonic,
        'predictions_original': test_preds,
        'predictions_platt': test_preds_platt,
        'predictions_isotonic': test_preds_isotonic,
        'true_labels': test_labels
    }


def plot_calibration_curve(y_true, y_prob_dict, model_name):
    """Plot calibration curve"""
    plt.figure(figsize=(10, 8))
    
    colors = ['blue', 'red', 'green']
    methods = list(y_prob_dict.keys())
    
    for i, (method, y_prob) in enumerate(y_prob_dict.items()):
        fraction_positives, mean_predicted = calibration_curve(
            y_true, y_prob, n_bins=10
        )
        plt.plot(mean_predicted, fraction_positives, 
                marker='o', color=colors[i], label=method, linewidth=2)
    
    # Perfect calibration line
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfect Calibration')
    
    plt.xlabel('Mean Predicted Probability')
    plt.ylabel('Fraction of Positives')
    plt.title(f'Calibration Curve - {model_name}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    
    plt.tight_layout()
    plt.savefig(f'{model_name}_calibration_curve.png', dpi=300, bbox_inches='tight')
    plt.show()


# ===== (9) Main pipeline =====
def main():
    # Fix random seed before any stochastic operation
    set_global_seed(SEED)

    X_tr, y_tr, X_cal, y_cal, X_val, y_val, X_te, y_te, input_dim = load_and_preprocess_data()

    # Compute positive-class weight
    pos_weight = (y_tr == 0).sum().item() / max(1, (y_tr == 1).sum().item())
    print(f"Positive-class weight pos_weight = {pos_weight:.2f}")

    # Sampler: make each batch more balanced (reproducible: fixed generator)
    y_tr_np = y_tr.numpy().astype(int)
    class_count = np.bincount(y_tr_np)
    class_weight = 1.0 / np.maximum(class_count, 1)
    sample_weight = class_weight[y_tr_np]

    # Unified CPU RNG (shared by sampler & dataloader)
    g_cpu = torch.Generator()
    g_cpu.manual_seed(SEED)

    sampler = WeightedRandomSampler(
        weights=torch.tensor(sample_weight, dtype=torch.double),
        num_samples=len(sample_weight),
        replacement=True,
        generator=g_cpu
    )

    batch_size = 128
    # For strongest reproducibility: num_workers=0 and pass the same generator
    train_loader = DataLoader(
        TensorDataset(X_tr, y_tr),
        batch_size=batch_size,
        sampler=sampler,
        drop_last=False,
        num_workers=0,
        generator=g_cpu,
        persistent_workers=False
    )
    
    cal_loader = DataLoader(
        TensorDataset(X_cal, y_cal),
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        generator=g_cpu,
        persistent_workers=False
    )
    
    val_loader = DataLoader(
        TensorDataset(X_val, y_val),
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        generator=g_cpu,
        persistent_workers=False
    )
    
    test_loader = DataLoader(
        TensorDataset(X_te, y_te),
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        generator=g_cpu,
        persistent_workers=False
    )

    # Instantiate models after setting randomness (their parameter initialization is controlled by the global seed)
    models_config = {
        'MLP': MLPModel(input_dim, hidden_dims=[512, 256, 128, 64, 32, 16], dropout=0.3),
        'GRU': GRUAttnModel(input_dim, emb_dim=64, hidden_dim=128, num_layers=2, bidirectional=True, dropout=0.3),
        'Transformer': TransformerModel(input_dim, d_model=32, nhead=8, num_layers=2, dropout=0.3)
    }

    results = {}
    for name, model in models_config.items():
        print("\n" + "="*50)
        print(f"Training {name} model")
        print("="*50)
        
        # Train model
        trained, best_val_auc, train_losses, val_aucs = train_model(
            model, train_loader, val_loader, epochs=120, lr=1e-3, pos_weight=pos_weight
        )
        
        # Evaluate model and perform calibration
        eval_results = evaluate_model_with_calibration(trained, cal_loader, test_loader, name)
        
        # Print results
        print(f"\n{name} Results:")
        print(f"Best validation AUC: {best_val_auc:.4f}")
        print(f"Test AUC (original): {eval_results['auc_original']:.4f}")
        print(f"Test AUC (Platt): {eval_results['auc_platt']:.4f}")
        print(f"Test AUC (Isotonic): {eval_results['auc_isotonic']:.4f}")
        
        print(f"\nCalibration metrics:")
        print(f"ECE (original): {eval_results['calibration_original']['ece']:.4f}")
        print(f"ECE (Platt): {eval_results['calibration_platt']['ece']:.4f}")
        print(f"ECE (Isotonic): {eval_results['calibration_isotonic']['ece']:.4f}")
        
        print(f"Brier Score (original): {eval_results['calibration_original']['brier_score']:.4f}")
        print(f"Brier Score (Platt): {eval_results['calibration_platt']['brier_score']:.4f}")
        print(f"Brier Score (Isotonic): {eval_results['calibration_isotonic']['brier_score']:.4f}")

        # Classification report
        y_hat_bin_original = (eval_results['predictions_original'] > 0.5).astype(int)
        y_hat_bin_platt = (eval_results['predictions_platt'] > 0.5).astype(int)
        y_hat_bin_isotonic = (eval_results['predictions_isotonic'] > 0.5).astype(int)
        
        print(f"\nClassification report (original):")
        print(classification_report(eval_results['true_labels'], y_hat_bin_original, digits=4))
        
        # Plot calibration curve
        y_prob_dict = {
            'Original': eval_results['predictions_original'],
            'Platt Scaling': eval_results['predictions_platt'],
            'Isotonic Regression': eval_results['predictions_isotonic']
        }
        plot_calibration_curve(eval_results['true_labels'], y_prob_dict, name)

        results[name] = {
            'model': trained,
            'val_auc': best_val_auc,
            'eval_results': eval_results,
            'train_losses': train_losses,
            'val_aucs': val_aucs
        }

    # Summary of results
    print("\n" + "="*80)
    print("Final Results Summary")
    print("="*80)
    print(f"{'Model':<12} {'Val AUC':<10} {'Test AUC (Original)':<20} {'Test AUC (Platt)':<18} {'Test AUC (Isotonic)':<21} {'ECE (Original)':<15} {'ECE (Platt)':<13} {'ECE (Isotonic)':<16}")
    print("-" * 120)
    
    for name in results:
        eval_res = results[name]['eval_results']
        print(f"{name:<12} {results[name]['val_auc']:<10.4f} {eval_res['auc_original']:<20.4f} {eval_res['auc_platt']:<18.4f} {eval_res['auc_isotonic']:<21.4f} "
              f"{eval_res['calibration_original']['ece']:<15.4f} {eval_res['calibration_platt']['ece']:<13.4f} {eval_res['calibration_isotonic']['ece']:<16.4f}")

    # Best model (based on original AUC)
    best_model_name = max(results.keys(), key=lambda k: results[k]['eval_results']['auc_original'])
    best_auc = results[best_model_name]['eval_results']['auc_original']
    print(f"\n🏆 Best model: {best_model_name} (AUC = {best_auc:.4f})")

    # Visualize training process
    plt.figure(figsize=(15, 5))
    
    # Training loss
    plt.subplot(1, 3, 1)
    for name in results:
        plt.plot(results[name]['train_losses'], label=name)
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.legend()

    # Validation AUC
    plt.subplot(1, 3, 2)
    for name in results:
        plt.plot(results[name]['val_aucs'], label=name)
    plt.title('Validation AUC')
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.grid(True)
    plt.legend()

    # Test-set AUC comparison (before/after calibration)
    plt.subplot(1, 3, 3)
    names = list(results.keys())
    x = np.arange(len(names))
    width = 0.25

    original_aucs = [results[n]['eval_results']['auc_original'] for n in names]
    platt_aucs = [results[n]['eval_results']['auc_platt'] for n in names]
    isotonic_aucs = [results[n]['eval_results']['auc_isotonic'] for n in names]

    plt.bar(x - width, original_aucs, width, label='Original', alpha=0.8)
    plt.bar(x, platt_aucs, width, label='Platt Scaling', alpha=0.8)
    plt.bar(x + width, isotonic_aucs, width, label='Isotonic Regression', alpha=0.8)

    plt.xlabel('Models')
    plt.ylabel('AUC')
    plt.title('Test-set AUC Comparison')
    plt.xticks(x, names)
    plt.legend()
    plt.ylim(0.7, 1.0)  # Adjust y-axis range to better see differences

    # Add value labels
    for i, (orig, platt, iso) in enumerate(zip(original_aucs, platt_aucs, isotonic_aucs)):
        plt.text(i - width, orig + 0.01, f"{orig:.3f}", ha='center', va='bottom', fontsize=8)
        plt.text(i, platt + 0.01, f"{platt:.3f}", ha='center', va='bottom', fontsize=8)
        plt.text(i + width, iso + 0.01, f"{iso:.3f}", ha='center', va='bottom', fontsize=8)

    plt.tight_layout()
    plt.show()

    return results, best_model_name


if __name__ == "__main__":
    results, best_model = main()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.calibration import calibration_curve
from sklearn.metrics import (brier_score_loss, average_precision_score, 
                            confusion_matrix, f1_score, precision_recall_curve, 
                            auc, roc_curve)

def calculate_youden_threshold(y_true, y_pred_proba):
    """
    Compute the optimal threshold based on Youden's index.
    Youden's index = TPR + TNR - 1 = Sensitivity + Specificity - 1
    """
    # Compute ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    
    # Compute TNR (specificity) = 1 - FPR
    tnr = 1 - fpr
    
    # Compute Youden's index
    youden_scores = tpr + tnr - 1
    
    # Find threshold corresponding to the maximum Youden's index
    optimal_idx = np.argmax(youden_scores)
    optimal_threshold = thresholds[optimal_idx]
    optimal_youden = youden_scores[optimal_idx]
    
    return optimal_threshold, optimal_youden, tpr[optimal_idx], tnr[optimal_idx]

# Read ground-truth labels
y_true = pd.read_csv("mimicdeadtest.csv")["death"].values

# Model files and names
model_files = {
    "MLP": "death_MLP_calibrated_proba.csv",
    "GRU": "death_GRU_calibrated_proba.csv", 
    "Transformer": "death_Transformer_calibrated_proba.csv"
}

# Create figure
plt.figure(figsize=(20, 12))

# Subplot 1: Calibration curves
plt.subplot(2, 3, 1)
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

# Store evaluation results for all models
evaluation_results = {}

# For each model, plot calibration curve and compute metrics
for i, (model_name, file) in enumerate(model_files.items()):
    y_pred = pd.read_csv(file).iloc[:, -2].values  # take the last predicted-probability column
    
    # Normalize probabilities (divide by 100 if >1)
    y_pred = [p * 0.01 if p > 1 else p for p in y_pred]
    y_pred = np.array(y_pred)
    
    # Compute calibration curve
    fraction_of_positives, mean_predicted_value = calibration_curve(
        y_true, y_pred, n_bins=10, strategy='quantile'
    )
    
    # Get optimal threshold using Youden's index
    optimal_threshold, youden_score, optimal_tpr, optimal_tnr = calculate_youden_threshold(y_true, y_pred)
    
    # Compute metrics based on the optimal threshold
    # 1. Brier score
    brier_score = brier_score_loss(y_true, y_pred)
    
    # 2. AUPRC (Area Under Precision-Recall Curve)
    auprc = average_precision_score(y_true, y_pred)
    
    # 3. Use the optimal threshold to compute TPR, TNR, F1
    y_pred_binary = (y_pred >= optimal_threshold).astype(int)
    
    # Compute confusion matrix for verification
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_binary).ravel()
    
    # Recompute TPR and TNR to ensure consistency
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    # F1 score
    f1 = f1_score(y_true, y_pred_binary)
    
    # Store results
    evaluation_results[model_name] = {
        'AUPRC': auprc,
        'TPR': tpr,
        'TNR': tnr,
        'Brier': brier_score,
        'F1': f1,
        'Threshold': optimal_threshold,
        'Youden': youden_score,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    }
    
    # Plot calibration curve
    plt.plot(mean_predicted_value, fraction_of_positives, "s-",
             label=f'{model_name} (Brier={brier_score:.3f})', 
             color=colors[i], linewidth=2, markersize=6)

# Plot the perfect calibration diagonal
plt.plot([0, 1], [0, 1], "k--", label="Perfect calibration", linewidth=1)

# Set calibration plot properties
plt.xlabel("Mean predicted probability", fontsize=12)
plt.ylabel("Fraction of positives", fontsize=12)
plt.title("Calibration curves for hemorrhagic shock prediction models", fontsize=14, fontweight='bold')
plt.legend(loc="upper left", fontsize=10)
plt.grid(True, alpha=0.3)
plt.xlim([0, 1])
plt.ylim([0, 1])

# Subplot 2: Precision-Recall curve
plt.subplot(2, 3, 2)
for i, (model_name, file) in enumerate(model_files.items()):
    y_pred = pd.read_csv(file).iloc[:, -2].values
    y_pred = [p * 0.01 if p > 1 else p for p in y_pred]
    y_pred = np.array(y_pred)
    
    # Compute Precision-Recall curve
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    auprc = evaluation_results[model_name]['AUPRC']
    
    plt.plot(recall, precision, color=colors[i], linewidth=2,
             label=f'{model_name} (AUPRC={auprc:.3f})')

plt.xlabel("Recall (TPR)", fontsize=12)
plt.ylabel("Precision", fontsize=12)
plt.title("Precision-Recall Curve", fontsize=14, fontweight='bold')
plt.legend(loc="upper right", fontsize=10)
plt.grid(True, alpha=0.3)
plt.xlim([0, 1])
plt.ylim([0, 1])

# Subplot 3: ROC curve and Youden's index
plt.subplot(2, 3, 3)
for i, (model_name, file) in enumerate(model_files.items()):
    y_pred = pd.read_csv(file).iloc[:, -1].values
    y_pred = [p * 0.01 if p > 1 else p for p in y_pred]
    y_pred = np.array(y_pred)
    
    # Compute ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, color=colors[i], linewidth=2,
             label=f'{model_name} (AUC={roc_auc:.3f})')
    
    # Mark the optimal Youden point
    optimal_threshold = evaluation_results[model_name]['Threshold']
    optimal_tpr = evaluation_results[model_name]['TPR']
    optimal_fpr = 1 - evaluation_results[model_name]['TNR']
    
    plt.plot(optimal_fpr, optimal_tpr, 'o', color=colors[i], markersize=8,
             markerfacecolor='white', markeredgewidth=2)

plt.plot([0, 1], [0, 1], "k--", label="Random classifier", linewidth=1)
plt.xlabel("False Positive Rate (1-TNR)", fontsize=12)
plt.ylabel("True Positive Rate (TPR)", fontsize=12)
plt.title("ROC Curves (dots mark Youden's optimal points)", fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, alpha=0.3)
plt.xlim([0, 1])
plt.ylim([0, 1])

# Subplot 4: Threshold comparison
plt.subplot(2, 3, 4)
model_names = list(evaluation_results.keys())
thresholds = [evaluation_results[model]['Threshold'] for model in model_names]
youden_scores = [evaluation_results[model]['Youden'] for model in model_names]

bars = plt.bar(model_names, thresholds, color=colors[:len(model_names)], alpha=0.7)
plt.xlabel("Models", fontsize=12)
plt.ylabel("Optimal threshold", fontsize=12)
plt.title("Optimal thresholds based on Youden's index", fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, (bar, threshold, youden) in enumerate(zip(bars, thresholds, youden_scores)):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{threshold:.3f}\n(Y={youden:.3f})', 
             ha='center', va='bottom', fontsize=10)

# Subplot 5: Metric comparison
plt.subplot(2, 3, 5)
metrics = ['AUPRC', 'TPR', 'TNR', 'F1']
x = np.arange(len(model_names))
width = 0.2

for i, metric in enumerate(metrics):
    values = [evaluation_results[model][metric] for model in model_names]
    plt.bar(x + i*width, values, width, label=metric, alpha=0.8)

plt.xlabel("Models", fontsize=12)
plt.ylabel("Score", fontsize=12)
plt.title("Metric comparison", fontsize=14, fontweight='bold')
plt.xticks(x + width*1.5, model_names)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3, axis='y')
plt.ylim([0, 1])

# Subplot 6: Brier score comparison (lower is better)
plt.subplot(2, 3, 6)
brier_scores = [evaluation_results[model]['Brier'] for model in model_names]
bars = plt.bar(model_names, brier_scores, color=colors[:len(model_names)], alpha=0.7)
plt.xlabel("Models", fontsize=12)
plt.ylabel("Brier score", fontsize=12)
plt.title("Brier score comparison (lower is better)", fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, score in zip(bars, brier_scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
             f'{score:.3f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

# Print detailed evaluation results
print("=" * 100)
print("                          Summary of model performance (optimal thresholds via Youden's index)")
print("=" * 100)
print(f"{'Model':<12} {'AUPRC':<8} {'TPR':<8} {'TNR':<8} {'Brier':<8} {'F1':<8} {'Threshold':<8} {'Youden':<8}")
print("-" * 100)

for model_name, results in evaluation_results.items():
    print(f"{model_name:<12} {results['AUPRC']:<8.3f} {results['TPR']:<8.3f} "
          f"{results['TNR']:<8.3f} {results['Brier']:<8.3f} {results['F1']:<8.3f} "
          f"{results['Threshold']:<8.3f} {results['Youden']:<8.3f}")

# Detailed confusion matrices
print("\n" + "=" * 100)
print("                              Detailed confusion matrices")
print("=" * 100)
for model_name, results in evaluation_results.items():
    print(f"\n{model_name} model (threshold={results['Threshold']:.3f}):")
    print(f"  True positives (TP): {results['TP']:4d}  |  False positives (FP): {results['FP']:4d}")
    print(f"  False negatives (FN): {results['FN']:4d}  |  True negatives (TN): {results['TN']:4d}")
    print(f"  Sensitivity (TPR): {results['TPR']:.3f}  |  Specificity (TNR): {results['TNR']:.3f}")
    print(f"  Youden's index: {results['Youden']:.3f}")

# Metric descriptions
print("\n" + "=" * 100)
print("                                  Metric descriptions")
print("=" * 100)
print("• AUPRC  : Area Under Precision-Recall Curve (higher is better, range 0-1)")
print("• TPR    : True Positive Rate / Sensitivity / Recall (higher is better, range 0-1)")  
print("• TNR    : True Negative Rate / Specificity (higher is better, range 0-1)")
print("• Brier  : Brier Score (lower is better, range 0-1)")
print("• F1     : F1 Score (higher is better, range 0-1)")
print("• Threshold : Optimal probability threshold based on Youden's index")
print("• Youden : Youden's index = TPR + TNR - 1 (higher is better, range 0-1)")

# Additional statistics
print("\n" + "=" * 100)
print("                               Basic dataset information")
print("=" * 100)
print(f"Total samples: {len(y_true)}")
print(f"Positives: {np.sum(y_true)} ({np.mean(y_true):.1%})")
print(f"Negatives: {len(y_true) - np.sum(y_true)} ({1-np.mean(y_true):.1%})")

# Find the best models
print("\n" + "=" * 100)
print("                                Best model picks")
print("=" * 100)

best_models = {
    'AUPRC': max(evaluation_results.items(), key=lambda x: x[1]['AUPRC']),
    'F1': max(evaluation_results.items(), key=lambda x: x[1]['F1']),
    'Brier': min(evaluation_results.items(), key=lambda x: x[1]['Brier']),
    'TPR': max(evaluation_results.items(), key=lambda x: x[1]['TPR']),
    'TNR': max(evaluation_results.items(), key=lambda x: x[1]['TNR']),
    'Youden': max(evaluation_results.items(), key=lambda x: x[1]['Youden'])
}

for metric, (model_name, results) in best_models.items():
    print(f"• Best {metric} model: {model_name} ({metric}={results[metric]:.3f})")

# Advantages of Youden's index method
print("\n" + "=" * 100)
print("                             Advantages of the Youden's index method")
print("=" * 100)
print("✓ Automatically finds the threshold that balances sensitivity and specificity for each model")
print("✓ Avoids issues with a fixed threshold (e.g., 0.5) that may not suit the actual data distribution")  
print("✓ Youden's index = TPR + TNR - 1; maximizing it jointly optimizes sensitivity and specificity")
print("✓ Especially suitable for medical diagnostics where both false positives and false negatives matter")
